## Vamos a hacer un fine-tuning de un modelo muy simple text-to-text con datos de la cafetería de la UAM

T5 (Text-to-Text Transfer Transformer) es un modelo preentrenado por Google que ya ha sido entrenado en miles de millones de frases (Wikipedia, libros, páginas web…)

La parte de aprender a entender el lenguaje y a generar texto coherente ya está definida en los parámetros de este modelo.

https://huggingface.co/docs/transformers/en/model_doc/t5

## ¿Qué es el fine-tuning entonces?

La idea principal es enseñarle al modelo una "nueva materia", donde busca patrones en un dataset para "aprender" a responder preguntas similares.

Podríamos decir que el modelo sabe "hablar" pero necesita aprender sobre "qué hablar".


<img src="https://github.com/GDG-UAM/workshop-huggingface/blob/main/notebook_final/fine_tuning.png?raw=1" alt="FT" width="700"/>


## ¿Como hacemos el fine-tuning?

Estos modelos están entrenados en redes neuronales, que naturalmente no entienden texto, por lo que hay que tokenizar el texto, es decir, convertirlos en números que pueden entender las redes neuronales.

Por ejemplo:

```python
"¿Cuánto cuesta el café?" → [1432, 209, 8743, 22, 1567, 1]
```


<img src="https://github.com/GDG-UAM/workshop-huggingface/blob/main/notebook_final/token.png?raw=1" alt="tk" width="700"/>


Una vez tokenizado nuestro dataset, ya se puede empezar el fine-tuning del modelo. La idea general es que el modelo aprende las relaciones que hay entre las preguntas y respuestas.

Estas relaciones se van grabando en los millones de parámetros que tiene, son pesos internos que minimizan el error de lo que genera y lo que debría generar.

## ¿Cómo genera una respuesta?

Cuando luego le haces una pregunta nueva como:
```markdown
¿Hay opciones sin gluten?
```
El modelo:

- Tokeniza tu texto.
- Pasa los tokens por el transformer.
- Genera tokens de salida uno a uno, prediciendo la palabra siguiente más probable.
- Decodifica esos tokens a texto.

Lo que nos da un resultado como:
```markdown
"Sí, tenemos galletas y bocadillos sin gluten."
```

## ¿Qué son los transformers?


<img src="https://github.com/GDG-UAM/workshop-huggingface/blob/main/notebook_final/transformers.png?raw=1" alt="Transformers" width="700"/>


https://huggingface.co/docs/transformers/en/index

La librería transofrmers es la clave de todo este trabajo, que se basan en un concepto llamado atención (attention).

La idea es que el modelo no procesa las frases palabra por palabra en orden,
sino que aprende a prestar atención a las partes más relevantes de la entrada para cada palabra que genera.

Es decir:

Para responder `“1,20€”` a `“¿Cuánto cuesta el café con leche?”`,
el modelo aprende que la palabra `“cuesta”` y `“café con leche”` son las claves,
y no necesita `“¿”` o `“el”`.


De esta forma los trasnformers manejan mejor el lenguaje natural en estos casos, ya que se se centran en manejar el contexto.

In [1]:
!git clone https://github.com/GDG-UAM/workshop-huggingface.git

Cloning into 'workshop-huggingface'...
remote: Enumerating objects: 60, done.
remote: Counting objects: 100% (60/60), done.
remote: Compressing objects: 100% (49/49), done.
remote: Total 60 (delta 19), reused 38 (delta 6), pack-reused 0 (from 0)
Receiving objects: 100% (60/60), 2.36 MiB | 19.50 MiB/s, done.
Resolving deltas: 100% (19/19), done.


In [24]:
%cd "workshop-huggingface/notebook_final"

/content/workshop-huggingface/notebook_final


In [25]:
%ls

cafeteria.json   llm_cafeteria.ipynb  transformers.png
fine_tuning.png  token.png


In [ ]:
!pip3 install "transformers[torch]" sentencepiece datasets pandas accelerate evaluate

In [ ]:
import json
import torch
from transformers import T5ForConditionalGeneration, T5Tokenizer, Trainer, TrainingArguments
from torch.utils.data import Dataset

- json → para leer tu dataset en formato JSON.
- torch → base de PyTorch, necesario para el modelo y entrenamiento.
- T5ForConditionalGeneration → modelo T5 preentrenado para generación de texto.
- T5Tokenizer → convierte texto en tokens (números que entiende el modelo).
- Trainer y TrainingArguments → sistema de entrenamiento de Hugging Face, facilita mucho el proceso.
- Dataset → estructura de datos personalizada de PyTorch.

## Cargamos nuestro dataset
Usaremos cafeteria.json que tiene datos sobre el menú de todas las cafeterías de la UAM.
Contiene pares de pregunta y respuesta.

In [ ]:
# Cargar archivo cafeteria.json
with open("cafeteria.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# Verificar primeras entradas
data[:3]

## Creamos la clase de CafeteriaDataset de PyTorch

PyTorch es un framework de deep learning, mientras Dataset es una clase abstracta de la que se hereda para crear datasets personalizados, permitiendo que el modelo acceda a datos de forma eficiente durante el entrenamiento (como en batches).

Convertimos las preguntas y respuestas en tensores numéricos que T5 pueda procesar.
Esto se hace con un tokenizador, que traduce texto a secuencias de números.

In [ ]:
class CafeteriaDataset(Dataset):
    def __init__(self, data, tokenizer, max_input=64, max_output=64):
        self.data = data
        self.tokenizer = tokenizer
        self.max_input = max_input
        self.max_output = max_output

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        question = self.data[idx]["question"]
        answer = self.data[idx]["answer"]

        # Tokenizar input (pregunta)
        input_enc = self.tokenizer(
            "Pregunta: " + question,
            truncation=True,
            padding="max_length",
            max_length=self.max_input,
            return_tensors="pt",
        )

        # Tokenizar target (respuesta)
        target_enc = self.tokenizer(
            answer,
            truncation=True,
            padding="max_length",
            max_length=self.max_output,
            return_tensors="pt",
        )

        return {
            "input_ids": input_enc["input_ids"].squeeze(),
            "attention_mask": input_enc["attention_mask"].squeeze(),
            "labels": target_enc["input_ids"].squeeze(),
        }

- input_ids → tokens de la pregunta (entrada al modelo).
- labels → tokens de la respuesta (lo que el modelo debe aprender a generar).
- attention_mask → indica qué partes son reales y cuáles son padding (relleno).
- El modelo aprende a generar la respuesta correcta dada una pregunta.

## Preparamos el modelo y tokenizamos

Ahora debemos elegir un modelo exitente, en eeste caso elegimos t5-small, es un modelo “text-to-text”: convierte texto de entrada en texto de salida.
Aquí lo cargamos junto con su tokenizador.

In [ ]:
model_name = "t5-small"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

- t5-small → versión ligera de T5 (más rápida para demos).
- tokenizer → convierte texto a tokens

Para ususarios de Apple se utiliza MPS, si no se utiliza la cpu

In [ ]:
# Detectar GPU Apple (MPS) o usar CPU
device = torch.device("mps") if torch.backends.mps.is_available() else torch.device("cpu")
print(f"Usando dispositivo: {device}")
model.to(device)

## Preparamos el dataset

In [ ]:
dataset = CafeteriaDataset(data, tokenizer)

## Configuramos el entrenamiento

In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=2,  # pequeño para demo
    num_train_epochs=10,
    logging_steps=5,
    save_total_limit=1,
    remove_unused_columns=False,  # importante para T5
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    tokenizer=tokenizer,
)

- num_train_epochs=10 → el modelo pasa 10 veces por todo el dataset.
- batch_size=2 → número de ejemplos por paso (puedes aumentarlo si tienes más RAM).
- Trainer → se encarga de todo: feed-forward, backpropagation, optimización, etc.

## Ahora entrenamos

In [ ]:
trainer.train();

## Guardamos el modelo

In [ ]:
model.save_pretrained("./modelo_cafeteria_t5")
tokenizer.save_pretrained("./modelo_cafeteria_t5")
print("Modelo entrenado y guardado en ./modelo_cafeteria_t5")

## Nuestro modelo ya estaría listo y entrenado ahora solo haría falta probarlo de la siguiente forma:

- Cargamos el modelo que se ha creado
- Lo probamos con la siguiente función

In [ ]:
def cargar_modelo_y_responder(model_path="./modelo_cafeteria_t5"):
    """
    Carga el modelo T5 entrenado y devuelve una función para responder preguntas.
    """
    # Cargar modelo y tokenizer
    tokenizer = T5Tokenizer.from_pretrained(model_path)
    model = T5ForConditionalGeneration.from_pretrained(model_path)

    device = torch.device("mps") if torch.backends.mps.is_available() else torch.device("cpu")
    model.to(device)
    model.eval()

    def responder(pregunta):
        input_enc = tokenizer(
            "Pregunta: " + pregunta,
            return_tensors="pt",
            truncation=True,
            padding="max_length",
            max_length=64,
        )
        input_enc = {k: v.to(device) for k, v in input_enc.items()}
        outputs = model.generate(
            **input_enc,
            max_new_tokens=64,
            num_beams=2,
            early_stopping=True,
        )
        return tokenizer.decode(outputs[0], skip_special_tokens=True)

    return responder

Pruebas:

In [ ]:
responder = cargar_modelo_y_responder()

print(responder("¿Cuánto cuesta el café con leche?"))
print(responder("¿Cuál es el precio del bocadillo de calamares?"))

El café con leche cuesta 1,05€.
El bocadillo de calamares cuesta 1,15€.
